### BUSINESS CHALLENGE
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential clients
We will be providing a company name and their primary website.
Here we are doing in 2 ways:
1. With OpenAI frontier model
2. With Ollama local server (For this you should already have ollama up and running in your local) (you can follow the steps from [here](./Text_Summarization_Using_Llama.ipynb))


In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI


In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

# MODEL = 'gpt-4o-mini'
MODEL = 'gpt-5-nano'
openai = OpenAI()

# If you want to use Ollama, you can uncomment the following line and comment the above line and comment the above 2 lines for OpenAI
# MODEL = "llama3.2"
# openai = OpenAI(base_url = 'http://localhost:11434/v1', api_key='ollama')



API key looks good so far


In [ ]:
# Let's just make sure the model is loaded, run below code if you are using Ollama

!ollama pull llama3.2

In [4]:
links = fetch_website_links('https://edwarddonner.com')
# links

In [5]:
contents = fetch_website_contents('https://edwarddonner.com')
# print(contents)


## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. I will cover this later.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
# print(get_links_user_prompt('https://edwarddonner.com'))

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [9]:
# select_relevant_links("https://edwarddonner.com")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [10]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [11]:
# print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [12]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
# get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL, # or MODEL if you are using Ollama otherwise use gpt-4.1-mini or gpt-5-nano
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: The AI community building the future

Hugging Face is a collaborative platform where the machine learning community creates, shares, and deploys models, datasets, and applications. With a strong open-source foundation, we help accelerate ML from research to real-world impact.

---

## What we offer

- A thriving collaboration hub for unlimited public models, datasets, and applications.
- Scale and speed with the HF open-source stack.
- Access to a diverse, multi-modal ecosystem: text, image, video, audio, and even 3D.
- Build and showcase your ML portfolio by sharing work with the global community.
- Enterprise-grade options: paid compute and dedicated solutions for teams and organizations.

- Global reach and growth: browse 1M+ models, 250k+ datasets, and 400k+ applications.
- Explore prebuilt AI apps and spaces to run, test, and deploy quickly.

---

## Our platform components

- Models: A vast collection of models across tasks and modalities (e.g., trending models like moonshotai/Kimi-K2-Thinking, maya-research/maya1, and many more).
- Datasets: Large and diverse datasets (e.g., nvidia/PhysicalAI-Autonomous-Vehicles, fka/awesome-chatgpt-prompts, Open-Bee/Honey-Data-15M, and more).
- Spaces: Hosted AI apps and demos you can run, customize, and extend (e.g., “The Smol Training Playbook,” image/video edit apps, DeepSite v3, and others).
- Community: A vibrant network of researchers, engineers, and practitioners sharing tools, know-how, and成果.
- Docs & Enterprise: Comprehensive documentation and enterprise-grade offerings to accelerate ML workflows.
- Pricing & Sign Up: Flexible plans to fit individual creators, teams, and organizations.

---

## Our culture and community

- The AI community building the future: open collaboration, transparency, and a shared drive to democratize AI.
- Open-source first: leveraging a robust HF open-source stack to move faster and more inclusively.
- All modalities and applications: tools and models spanning text, image, video, audio, and 3D.
- A platform for you to shine: build, publish, and grow your ML profile by sharing your work with the world.

---

## Customers, use cases, and impact

- Organizations and individuals leverage Hugging Face to develop, evaluate, and deploy ML models with ease.
- Use cases span research, industry applications, and creative apps, from NLP to computer vision and multimodal projects.
- Examples you can explore on the platform include training dashboards, model decks, image edit workflows, and novel AI applications built with Spaces.
- We support teams with enterprise compute and services to accelerate ML projects at scale.

---

## Careers at Hugging Face

- A growing team focused on ML, NLP, and deep learning, with opportunities to influence the AI landscape.
- Company snapshot: founded in 2016, privately held, based in Paris, with a fast-growing global community (staff sizes in the range of a few dozen to a couple hundred).
- Open roles and current openings are listed on our Careers page. If you’re excited about open science, collaboration, and building tools that democratize AI, Hugging Face could be a fit.
- See jobs and learn more about joining our team via the Careers page: Hugging Face – Current Openings.

---

## Get started

- Visit hugginface.co to explore models, datasets, spaces, and docs.
- Sign up to join the community, publish your work, and access enterprise options when you’re ready to scale.
- For career opportunities, check the Careers page for current openings.

Hugging Face is the AI community building the future—join us to create, share, and deploy the next generation of AI together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL, # or MODEL if you are using Ollama otherwise use gpt-4.1-mini or gpt-5-nano
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: The AI community building the future

Hugging Face is the platform where the machine learning community collaborates on models, datasets, and applications. We’re on a mission to solve and democratize artificial intelligence through natural language, and we’re powered by an open, collaborative spirit.

## What we do
- Create, discover, and collaborate on ML projects with the HF open source stack.
- Host and share unlimited public models, datasets, and applications.
- Support all modalities: text, image, video, audio, and even 3D.
- Help organizations move faster with enterprise-grade compute and deployments.

## Our Platform
- Models: Browse 1M+ models to find components for your next project.
- Datasets: Explore 250k+ datasets to train and evaluate your models.
- Spaces: Run, deploy, and share AI apps with an easy-to-use infrastructure (examples include real-time inference, image and video editing apps, and more).
- Community and Docs: Rich documentation and a global community to learn from and contribute to.
- Enterprise & Pricing: Paid compute and enterprise solutions for teams and organizations.
- Multimodal capabilities: Text, image, video, audio, and 3D workflows supported.

## Why it matters for customers
- Accelerate ML development: Access a vast ecosystem of models, datasets, and apps in one place.
- Collaborate openly: Share work with the world, get feedback, and build your ML portfolio.
- Scale with enterprise options: Compute resources and enterprise-grade solutions to accelerate production.

## For developers, researchers and builders
- Build your portfolio: Share your work publicly and showcase your ML profile.
- Explore all modalities: Work across text, images, video, audio, and 3D.
- Learn and contribute: Join a vibrant community that advances NLP, ML, and deep learning.

## Culture and community
- The AI community building the future: A collaborative, open, and inclusive culture that democratizes AI.
- Open source first: Move faster by leveraging and contributing to an open source stack.
- Global reach, Paris roots: Founded in 2016, Hugging Face is privately held with a base in Paris, France, growing a diverse team around the world.

## Careers and opportunities
- Current openings: Hugging Face publicly lists career opportunities; join a growing team shaping the future of AI.
- Company profile: Private, 51–200 employees, founded in 2016; specialties in machine learning, natural language processing, and deep learning.
- Get involved: Explore roles that align with research, engineering, product, and enterprise solutions.

## Quick facts
- Founded: 2016
- Location: Paris, FR (primary)
- Size: 51–200 employees
- Industry: Software Development
- Core mission: Solve and democratize artificial intelligence through natural language
- Key products: Hugging Face platform, datasets, models, and Spaces
- Notable capability: Browse 1M+ models, 250k+ datasets, and 400k+ applications

Explore, build, and collaborate with Hugging Face—the home of machine learning. If you’re a customer, a contributor, or a future teammate, your work can help push AI forward.